In [17]:
import pandas as pd

# Load the CSV file
file_path = '/Users/lebakuprathyushkumarreddy/Downloads/pavement_with_crashes_for_each_collisiontype_csvfile/merged_csv_file_using_geopandas.csv'
df = pd.read_csv(file_path)

data = df
# Grouping the data by 'OBJECTID' and counting the crashes
crash_counts = data.groupby('OBJECTID').size()
# Creating a new dataframe with OBJECTID and the crash counts
pavement_section_data = data.drop_duplicates(subset='OBJECTID').set_index('OBJECTID')
pavement_section_data['Crash_Count'] = crash_counts

# Resetting the index to include OBJECTID as a column
pavement_section_data.reset_index(inplace=True)

# The final dataframe is stored in pavement_section_data

pavement_section_data["Crash_Rate"]= pavement_section_data["Crash_Count"] *100000000/(pavement_section_data["AADT"]*pavement_section_data["PMIS_LENGTH"]*365)

print(f"Total crashes is {pavement_section_data['Crash_Count'].sum()} ")



Total crashes is 21536 


/var/folders/yl/h3jt_tns2s14c8y5w9qptb6c0000gn/T/ipykernel_31714/1261433241.py:5: DtypeWarning: Columns (143,172,176,177,178,179,188,203,204,207,208,209,210,211,212,243,244,252,257,258,267,268,274,275,281) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [18]:
iri_bins = [0, 95, 170, float('inf')]
iri_labels = ['Good', 'Fair', 'Poor']
pavement_section_data['IRI_Category'] = pd.cut(pd.to_numeric(pavement_section_data['IRI'], errors='coerce'), bins=iri_bins, labels=iri_labels)
mean_crash_rates = pavement_section_data.groupby('IRI_Category')['Crash_Rate'].mean() # to find mean crash rate in each segment
print("Mean crash rates per million vmt:\n", mean_crash_rates)

Mean crash rates per million vmt:
 IRI_Category
Good    134.123662
Fair    225.617394
Poor    468.304529
Name: Crash_Rate, dtype: float64


In [47]:
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import f_oneway

iri_bins = [0, 95, 170, float('inf')]
iri_labels = ['Good', 'Fair', 'Poor']
pavement_section_data['IRI_Category'] = pd.cut(pd.to_numeric(pavement_section_data['IRI'], errors='coerce'), bins=iri_bins, labels=iri_labels)

# Extracting crash rates for each IRI category
good_iri_crash_rates = pavement_section_data[pavement_section_data['IRI_Category'] == 'Good']['Crash_Rate'].dropna()
fair_iri_crash_rates = pavement_section_data[pavement_section_data['IRI_Category'] == 'Fair']['Crash_Rate'].dropna()
poor_iri_crash_rates = pavement_section_data[pavement_section_data['IRI_Category'] == 'Poor']['Crash_Rate'].dropna()

# ANOVA Test
anova_result = f_oneway(good_iri_crash_rates, fair_iri_crash_rates, poor_iri_crash_rates)
print(f"ANOVA Test: p-value = {anova_result.pvalue}")

# Proceed with post-hoc tests if ANOVA is significant
if anova_result.pvalue < 0.05:
    # Tukey-Kramer Test
    crash_rates = np.concatenate([good_iri_crash_rates, fair_iri_crash_rates, poor_iri_crash_rates])
    iri_categories = ['Good'] * len(good_iri_crash_rates) + ['Fair'] * len(fair_iri_crash_rates) + ['Poor'] * len(poor_iri_crash_rates)

    tukey_result = pairwise_tukeyhsd(endog=crash_rates, groups=iri_categories, alpha=0.05)
    print(tukey_result.summary())
    
    print()
    # Fisher's LSD Test
    fisher_lsd_results = {
        'Good-Fair': stats.ttest_ind(good_iri_crash_rates, fair_iri_crash_rates, equal_var=False),
        'Good-Poor': stats.ttest_ind(good_iri_crash_rates, poor_iri_crash_rates, equal_var=False),
        'Fair-Poor': stats.ttest_ind(fair_iri_crash_rates, poor_iri_crash_rates, equal_var=False)
    }

    print("Fisher's LSD Test Results:")
    for comparison, result in fisher_lsd_results.items():
        print(f"{comparison}: t-statistic = {result.statistic}, p-value = {result.pvalue}")
else:
    print("No significant differences found in ANOVA, post-hoc tests not required.")


ANOVA Test: p-value = 3.304575171596344e-88
 Multiple Comparison of Means - Tukey HSD, FWER=0.05  
group1 group2 meandiff p-adj   lower    upper   reject
------------------------------------------------------
  Fair   Good -91.4937   0.0 -114.4243 -68.5632   True
  Fair   Poor 242.6871   0.0  203.1637 282.2106   True
  Good   Poor 334.1809   0.0  295.5067 372.8551   True
------------------------------------------------------

Fisher's LSD Test Results:
Good-Fair: t-statistic = -9.549862098734893, p-value = 3.57238159195401e-21
Good-Poor: t-statistic = -13.912915260260002, p-value = 6.93844278612184e-35
Fair-Poor: t-statistic = -9.689958845947373, p-value = 5.1556135759091546e-20
